In [1]:
import pandas as pd
import numpy as np

In [23]:
org = pd.read_csv('control_task2.tsv', sep='\t')
fake = pd.read_csv('general_task2.tsv', sep='\t')
org['Race'] = None
fake['Race'] = None

In [25]:
cols = ['INPUT:image', 'Race']
org[cols].to_csv('org_label.csv', index=None)
fake[cols].to_csv('fake_label.csv', index=None)

## Task 2

In [2]:
result = pd.read_csv('assignments_from_pool_29960711__06-12-2021.tsv', sep='\t')

In [3]:
def link2id(x):
    return x.split('/')[-1].split('"')[0]

result['INPUT:image'] = result['INPUT:image'].apply(link2id)

In [4]:
result['GOLDEN:result'] = result['GOLDEN:result'].fillna(1.)

In [5]:
generated_img_result = result[result['GOLDEN:result'] == 1.0]

In [6]:
aggregated_result = generated_img_result.groupby('INPUT:image').sum()

In [7]:
final_result = aggregated_result[['OUTPUT:gender', 'OUTPUT:result', 'OUTPUT:confidence', 'GOLDEN:result']]

In [8]:
final_result = final_result.rename({
    'OUTPUT:gender': 'gender', 
    'OUTPUT:result': 'result',
    'OUTPUT:confidence': 'confidence', 
    'GOLDEN:result': 'n_workers'
}, axis=1)

In [9]:
final_result['n_workers'] = final_result['n_workers'].astype('int')

workers do not always agree on genders of the image: indicating random selections

In [80]:
def control_consensus(df):
    # male or female
    return sum((df['gender'] == df['n_workers']) | (df['gender'] == 0)) / len(df)

In [81]:
control_consensus(final_result)

0.912

images considered to be `real` by workers with more than 2 votings

In [82]:
# workers deem to be real
real_img = final_result[(final_result['n_workers'] - final_result['result'] >= 3)]

workers do not always agree with the gender

In [83]:
control_consensus(real_img)

0.9516129032258065

rate for successfully deceive more than 2 workers

In [84]:
n_real_imgs = len(real_img)
n_valid_imgs = len(final_result[final_result['n_workers'] >= 3])
f"decieved imgs: {n_real_imgs}; imgs with more than 3 workers: {n_valid_imgs} rate: {n_real_imgs / n_valid_imgs}"

'decieved imgs: 62; imgs with more than 3 workers: 424 rate: 0.14622641509433962'

In [36]:
f"concensus on gender: {control_consensus(real_img)}"

'concensus on gender: 0.9516129032258065'

### Create Task 3

In [47]:
real_img_result = result[result['GOLDEN:result'] == 0.]
real_img_result['n_workers'] = np.ones(len(real_img_result))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
aggregated_real_img_result = real_img_result.groupby('INPUT:image').sum()\
    [['OUTPUT:gender', 'OUTPUT:result', 'OUTPUT:confidence', 'n_workers']]

In [55]:
# workers deemed as fake
fake_img = aggregated_real_img_result[aggregated_real_img_result['OUTPUT:result'] >= 2]

In [56]:
task = pd.read_csv('tasks/tasks_for_pool_29747529__23-11-2021_training.tsv', sep='\t')

In [69]:
from random import random

new_data = task.iloc[0:0]
real_img_list = real_img.index.to_list()
fake_img_list = fake_img.index.to_list()

for i in range(len(real_img_list)):
    image_1 = "https://raw.githubusercontent.com/BirkhoffG/crowd_project/main/img/" + real_img_list[i]
    image_2 = "https://raw.githubusercontent.com/BirkhoffG/crowd_project/main/original/" + fake_img_list[i]
    if random() < 0.5:
        new_data = new_data.append({
            'INPUT:image_left': image_1, 'INPUT:image_right': image_2, 'GOLDEN:result': 'Right', 
            'GOLDEN:textfield': '', 'HINT:text': 'The image on the Right is REAL.'
        }, ignore_index=True)
    else:
        new_data = new_data.append({
            'INPUT:image_left': image_2, 'INPUT:image_right': image_1, 'GOLDEN:result': 'Left', 
            'GOLDEN:textfield': '', 'HINT:text': 'The image on the Left is REAL.'
        }, ignore_index=True)

In [70]:
new_data.to_csv('task3.tsv', sep='\t', index=None)

## Task 3

In [2]:
df_1 = pd.read_csv('assignments_from_pool_29778972__06-12-2021.tsv', sep='\t')

In [7]:
df_2 = pd.read_csv('assignments_from_pool_30134683__06-12-2021.tsv', sep='\t')

In [4]:
df_1.dropna(how='all', inplace=True)

In [9]:
df_2.dropna(how='all', inplace=True)

In [6]:
sum(df_1['OUTPUT:result'] == df_1['GOLDEN:result']) / len(df_1)

0.755

In [10]:
sum(df_2['OUTPUT:result'] == df_2['GOLDEN:result']) / len(df_2)

0.6483870967741936

In [11]:
df_2

,INPUT:image_left,INPUT:image_right,OUTPUT:result,OUTPUT:textfield,GOLDEN:result,GOLDEN:textfield,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started
0,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Right,NaN,Right,NaN,The image on the Right is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,c9373b05-46bb-4f17-9e0d-d82a4d32f37e,0001cbd19b--61ae7180e6166d6a31848125,9c0967fe626070766f09aa93ac682584,APPROVED,2021-12-06T20:24:32.133
2,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Right,NaN,Right,NaN,The image on the Right is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,df441413-2fb6-4c76-a7ab-ce014b441923,0001cbd19b--61ae7189b402c123889dab8d,9c0967fe626070766f09aa93ac682584,APPROVED,2021-12-06T20:24:41.857
4,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Left,NaN,Right,NaN,The image on the Right is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,41cab91a-1c71-4ce5-b4b8-56201b519260,0001cbd19b--61ae71920910fc6b97ec9deb,9c0967fe626070766f09aa93ac682584,APPROVED,2021-12-06T20:24:50.483
6,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Right,NaN,Left,NaN,The image on the Left is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,689e8281-6333-49ee-84cc-ca68d36d4fd4,0001cbd19b--61ae71a2ec7c4619a7c39d19,9c0967fe626070766f09aa93ac682584,APPROVED,2021-12-06T20:25:06.713
8,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Right,NaN,Left,NaN,The image on the Left is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,6b5105ca-a1e5-422c-8441-d18513911044,0001cbd19b--61ae71c67e288174a932d202,2a4f5f5ccd67527c17e3205408b104a6,APPROVED,2021-12-06T20:25:42.880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Left,NaN,Left,NaN,The image on the Left is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,b427e966-725e-455b-82a8-6ea479c7307f,0001cbd19b--61ae78041d7fbd5ee3b4464a,ca40f99e62bf88e3cd89786babbfc87e,APPROVED,2021-12-06T20:52:20.197
612,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Right,NaN,Right,NaN,The image on the Right is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,0dea1730-9277-483b-ad61-d00ccc375d9e,0001cbd19b--61ae780b127fe1158a524060,49acc01712d3578d346224507eda7fd1,APPROVED,2021-12-06T20:52:27.500
614,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Right,NaN,Right,NaN,The image on the Right is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,0dea1730-9277-483b-ad61-d00ccc375d9e,0001cbd19b--61ae780c6b8dbc6f8ec2f1e3,d0b27dac0f3789c72dd83e1d89c257d1,APPROVED,2021-12-06T20:52:28.990
616,https://raw.githubusercontent.com/BirkhoffG/cr...,https://raw.githubusercontent.com/BirkhoffG/cr...,Left,strange neck,Left,NaN,The image on the Left is REAL.,NaN,https://toloka.yandex.ru/task/30134683/0001cbd...,9ff930c8-5c4f-43fa-a563-ca845e963ede,0001cbd19b--61ae78c843766709674b5a9e,d8e61a54e640dc786924fbd68be12067,APPROVED,2021-12-06T20:55:36.190
